<a href="https://colab.research.google.com/github/lqcgithub/MiningofMassiveDatasets/blob/main/Spark_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
pip install pyspark

In [42]:
import pandas as pd
import time
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RS').getOrCreate()

In [43]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
metadata = spark.read.option("header",True).csv("/content/drive/MyDrive/sample-datasets/ratings.csv")

In [46]:
metadata.head(5)

[Row(userId='1', movieId='296', rating='5.0', timestamp='1147880044'),
 Row(userId='1', movieId='306', rating='3.5', timestamp='1147868817'),
 Row(userId='1', movieId='307', rating='5.0', timestamp='1147868828'),
 Row(userId='1', movieId='665', rating='5.0', timestamp='1147878820'),
 Row(userId='1', movieId='899', rating='3.5', timestamp='1147868510')]

In [47]:
metadata.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [48]:
metadata = metadata.withColumn("userId",col("userId").cast("int")).withColumn("movieId",col("movieId").cast("int")).withColumn("rating",col("rating").cast("float")).withColumn("timestamp",col("timestamp").cast("double"))

In [49]:
(training, test) = metadata.randomSplit([0.8, 0.2])

In [50]:
start = time.time()
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)
recomendations = model.recommendForAllUsers(10)
end = time.time()
print("ALS completed in {:.4f} seconds" .format(end-start))

ALS completed in 182.2160 seconds


In [51]:

recomendations.show(5)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[{177657, 11.4261...|
|   463|[{188225, 18.2845...|
|   471|[{176459, 12.5576...|
|   496|[{160557, 14.2247...|
|   833|[{177657, 12.6587...|
+------+--------------------+
only showing top 5 rows



In [52]:
predictions = model.transform(test)

In [53]:
predictions.show()

+------+-------+------+-------------+----------+
|userId|movieId|rating|    timestamp|prediction|
+------+-------+------+-------------+----------+
|124207|    148|   3.0| 8.47246861E8| 1.3863465|
|  8249|    148|   3.0| 8.35454413E8| 2.7297518|
|159730|    148|   3.0| 8.42162037E8| 2.4039497|
| 21531|    148|   3.0| 8.34035555E8|  2.691166|
| 54331|    148|   2.0| 9.54702916E8|  3.294952|
| 14831|    148|   3.0| 9.44148276E8| 2.7285526|
| 66907|    148|   4.0| 8.44353203E8| 3.8418221|
| 31171|    148|   2.0| 8.55003432E8| 2.4895992|
| 98520|    148|   4.0|1.034547175E9|  3.052722|
| 66440|    148|   2.5|1.099143605E9| 2.7752597|
| 49403|    148|   2.0|1.505584741E9| 1.5442522|
|135438|    148|   2.0| 9.45010495E8|  3.247229|
| 25664|    148|   3.0| 8.35559154E8| 3.1376905|
| 66358|    148|   3.5|1.064853335E9|  2.358512|
| 81091|    148|   1.0|   8.347245E8|  2.608543|
|132310|    148|   3.0| 8.36248537E8| 1.8244994|
|115095|    148|   4.0|  9.4493074E8| 3.1624346|
| 74598|    148|   3

In [59]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = nan
